In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/article-ii/paget-et-al-2023-platelet-rich-plasma-injections-for-the-treatment-of-ankle-osteoarthritis.pdf
/kaggle/input/article-1/AP_20211103_SONNERY_COTTET_Article publi.pdf
/kaggle/input/article-iv/gebhardt-et-al-2023-the-influence-of-arthroscopic-shaver-mincing-and-platelet-rich-plasma-on-chondrocytes-of.pdf
/kaggle/input/article-v/hollander-et-al-2019-adaptation-of-running-biomechanics-to-repeated-barefoot-running-a-randomized-controlled-study.pdf
/kaggle/input/article-iii/bramah-et-al-2024-the-sprint-mechanics-assessment-score-a-qualitative-screening-tool-for-the-in-field-assessment-of.pdf
/kaggle/input/synopsis-2/2024.05.29_Synopsis_v1.0_CAPSULAR_Format_HDH Old.docx
/kaggle/input/synopsis-1/2024.06.24_Synopsis_GODENECHE_Format_HDH_VF.docx


# **PDF PROCESS ARTICLE**

In [2]:
!pip install pypdf2
from PyPDF2 import PdfReader
import os
from pprint import pprint
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 828.5 kB/s eta 0:00:00


In [3]:
def PDF_PROCESS (pdf_path) :
    
    pages_text,Metadata,tab=extract_information2(pdf_path)
    Textab=tab
    label,text,start=Summary(pages_text)
    Title,Text=CORE(pages_text,start+1)
    SubTitle=[]
    SubText=[]
    for i in range (len(Title)):
        a,b=SUBCORE(Text[i])
        if len(a)!=0 :
            SubTitle.append(a)
            SubText.append(b)
        else :
            SubTitle.append([Title[i]])
            SubText.append([Text[i]])
    Content=[]
    Titab=[]
    for i in range (len(tab)) :
        Content.append(INFO_TABLE(tab[i])[0:3])
        Titab.append(INFO_TABLE(tab[i])[3])
        
    Parts={
        "SubTitle" : SubTitle,
        "Text" : SubText
    }


    Sum={
        "label" : label,
        "text" : text
    }

    Core={
        "Title" : Title,
        "Parts" : Parts
    }

    Table= {
        "Titab" :Titab,
        "Content" :Content,
        "Textab" :Textab
    }

    Document = {
        "Metadata": Metadata,
        "Sum": Sum,
        "Core": Core,
        "Table": Table
    }
    return(Document)



In [4]:
def extract_information2(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PdfReader(f)
        meta = reader.metadata
        number_of_pages = len(reader.pages)    
        pages_text = []
        for n in range(number_of_pages):
            page = reader.pages[n]
            pages_text.append('\n'+page.extract_text())
    metadata = f"""
    Information about {pdf_path}: 

    Author: {meta.author}
    Creator: {meta.creator}
    Producer: {meta.producer}
    Subject: {meta.subject}
    Title: {meta.title}
    Number of pages: {number_of_pages}
    """
    g=re.compile('TABLE ')
    print(metadata)
    bel,tab,pages_text=EXTRACT_TABLE2 (pages_text)
    return(pages_text,metadata,tab)

In [5]:
def EXTRACT_TABLE2 (pages_text) :
    i=0
    n=len(pages_text)
    res=[]
    label=[]
    p=re.compile('TABLE [0-9]\n')
    d=re.compile('([0-9]\n[^012345789]{40,100})|(TABLE [0-9]\n)')
    while i<n :
        pos=p.search(pages_text[i])
        if pos!=None :
            label.append(pages_text[i][pos.start():pos.end()])
            dos=d.search(pages_text[i][pos.end():])
            res.append(pages_text[i][pos.end():pos.end()+dos.start()])
            pages_text[i]=pages_text[i][:pos.start()]+pages_text[i][pos.end()+dos.start():]
        else :
            i+=1
    return(label,res,pages_text)

In [6]:
def CORE(pages_text,start) :
    res=[]
    label=["INTRODUCTION"]
    n=len(pages_text)
    i=0
    rep=0
    p=re.compile('([A-Z]{6,}\n)|(ORCID iDs)')
    Prems=0
    while i<n :
        pos=p.search(pages_text[i][rep:])
        if Prems==0 and pos!=None :
            doc=pages_text[0][start:]
            for l in range (i-1):
                doc+=pages_text[l]
            doc+=pages_text[i][:pos.start()+rep]
            Prems=1
            res.append(doc)
        if pos!=None :
            label.append(pages_text[i][pos.start()+rep:pos.end()+rep])
            start=pos.end()+rep
            rep=start
            j=i
            while(j<n) :
                pos=p.search(pages_text[j][rep*(i==j):])
                if pos!=None :
                    end=pos.start()+rep*(i==j)
                    doc=''
                    k=j-i
                    doc+=pages_text[i][start-1:(i==j)*(end-1)+(i!=j)*(len(pages_text[i]))]
                    for l in range(k-1) :
                        doc+=pages_text[i+l+1]
                    doc+=pages_text[j][:(end-1)*(i!=j)]
                    res.append(doc)
                    j=n+1
                j+=1
            if j==n :
                doc=pages_text[i][start:]
                for l in range(n-1-i) :
                    doc+=pages_text[i+l+1]
                res.append(doc)
                i=n
        else :
            i+=1
            rep=0
    return (label,res)

In [7]:
def Summary(pages_text) :
    Marks = ['Background:', 'Purpose/Hypothesis:','Purpose:','Hypothesis:','Study Design:','Methods', 'Results:',  'Conclusion:', 'Registration:','Keywords:','\n']
    res=[]
    label=[]
    for i in range (len(Marks)-1) :
        p=re.compile(Marks[i])
        pos=p.search(pages_text[0])
        if pos!=None :
            start=pos.end()
            label.append(Marks[i])
            j=int(1)
            while(j<len(Marks)) :
                p=re.compile(Marks[i+j])
                pos=p.search(pages_text[0][start:])
                if pos!=None :
                    end=pos.start()
                    res.append(pages_text[0][start:start + end-1])
                    j=len(Marks)
                j+=1
    return (label,res,start + end)

In [8]:
def SUBCORE(par) :
    res=[]
    label=['Introduction']
    p=re.compile('\n((?!Journal)[A-Z][\-a-z\, ]{1,20}\s*){1,}\n')
    pos=p.search(par)
    prems=0
    while pos!=None :
        prems+=1
        res.append(par[:pos.start()])
        label.append(par[pos.start():pos.end()])
        par=par[pos.end():]
        pos=p.search(par)   
    res.append(par)
    if prems==0 :
        label=['Text Body']
    return (label,res)

In [9]:
def INFO_TABLE (tab) :
        p=re.compile('[A-Z][a-z]{1,20}\s(\(*[A-Z][a-z\) ]{1,40}){1,}\n')
        pos=p.search(tab)
        titre = tab[pos.start():pos.end()]
        tab=tab[pos.end():]
        p=re.compile('([A-Z][a-z ]{1,40}){1,}[Nn=0-9\( ]*[\)\n]+')
        r=re.compile('\n')
        dos=r.search(tab)
        tabtit=tab[:dos.end()]
        tab=tab[dos.end():]
        pos = p.search(tabtit)
        Head=[]
        Line=[]
        nb=[]
        while pos!=None :
            Head.append(tabtit[pos.start():pos.end()])
            tabtit=tabtit[pos.end():]
            pos=p.search(tabtit)   
        p=re.compile('[0-9]')
        pos=p.search(tab)        
        while pos!=None :
            Line.append(tab[:pos.start()])
            tab=tab[pos.start():]
            p=re.compile('[A-Z\n]')
            pos=p.search(tab)
            if pos!=None :
                nb.append(EXTRACT_LINE(tab[:pos.start()],Head))
                tab=tab[pos.start():]
            else :
                nb.append(EXTRACT_LINE(tab,Head))
                tab=''
            p=re.compile('[0-9\.\(\) ]{5,}')
            pos=p.search(tab)
        return(Head,Line,nb,titre)

In [10]:
def EXTRACT_LINE(line,Head) :
    d=re.compile('[0-9]')
    pos=d.search(line)
    line=line[pos.start():]
    p=re.compile('\s')    
    pos=p.search(line)
    nb=[]
    while pos!=None :
        nb.append(line[:pos.start()])
        line=line[pos.end():]
        pos=p.search(line)
    if d.search(line)!=None :    
        nb.append(line)
    n=len(nb)
    res=[]
    if n == len(Head)*2 :
        for i in range (n//2) :
            if nb[2*i+1][0]=='(' :
                res.append(nb[2*i]+nb[2*i+1])
            else :
                res.append(nb[2*i]+'+-'+nb[2*i+1][1:])
    elif n==len(Head)*2-1 :
        for i in range (n//2) :
            if nb[2*i+1][0]=='(' :
                res.append(nb[2*i]+nb[2*i+1])
            else :
                res.append(nb[2*i]+'+-'+nb[2*i+1][1:])
        res.append(nb[n-1])
    else :
        res=nb
    return(res)


In [11]:
Document=PDF_PROCESS("/kaggle/input/article-v/hollander-et-al-2019-adaptation-of-running-biomechanics-to-repeated-barefoot-running-a-randomized-controlled-study.pdf")


    Information about /kaggle/input/article-v/hollander-et-al-2019-adaptation-of-running-biomechanics-to-repeated-barefoot-running-a-randomized-controlled-study.pdf: 

    Author: Karsten Hollander, Dominik Liebl, Stephanie Meining, Klaus Mattes, Steffen Willwacher, and Astrid Zech
    Creator: C&M
    Producer: Acrobat Distiller 8.0.0 (Windows); modified using iText 4.2.0 by 1T3XT
    Subject: Am J Sports Med 2019.47:1975-1983
    Title: Adaptation of Running Biomechanics to Repeated Barefoot Running: A Randomized Controlled Study
    Number of pages: 9
    


In [12]:
print("PART :", Document["Sum"]["label"][3])
print("CONTENT :",Document["Sum"]["text"][3])

PART : Study Design:
CONTENT :  Controlled laboratory study.


In [13]:
print("PART :", Document["Core"]["Title"][1])
print("SUBPART :",Document["Core"]["Parts"]["SubTitle"][1][1])
print("CONTENT :",Document["Core"]["Parts"]["Text"][1][1])

PART : METHODS

SUBPART : 
Study Design

CONTENT : This study was a prospective 3-arm randomized single-
blinded controlled study over 8 weeks, that was approvedby the local ethics committee (pr otocol ID37). All participants
granted their informed consent. Reporting of this study
adhered to the CONSORT statement
31(Consolidated Stand-
ards of Reporting Trials), and it was prospectively registeredin the German Clinical Trial Register (DRKS00011073).


In [14]:
print("First Table :", Document["Table"]["Titab"][0])
print("First Table :",Document["Table"]["Content"][0])


First Table : Pretest Data and Statistics for Barefoot and Shod Running Conditionsa

First Table : (['Value\n'], ['Foot strike index ', '\nAngle at ground contact\nAnkle –', '\nFoot –', '\nKnee'], [['0.27', '(0.24)', '0.07', '(0.05)', '\\.001'], ['4.0', '(11.0)', '–12.0', '(4.3)', '\\.001'], ['12.2', '(8.32)', '–22.4', '(4.5)', '\\.001'], ['8.7', '(6.0)', '6.2', '(6.1)', '\\.00']])


In [15]:
print(extract_information2('/kaggle/input/article-1/AP_20211103_SONNERY_COTTET_Article publi.pdf')[0])


    Information about /kaggle/input/article-1/AP_20211103_SONNERY_COTTET_Article publi.pdf: 

    Author: Charles Pioger, Lampros Gousopoulos, Graeme P. Hopper, Thais Dutra Vieira, Joao Pedro Campos, Abdo El Helou, Corentin Philippe, Adnan Saithna, and Bertrand Sonnery-Cottet
    Creator: C&M
    Producer: Acrobat Distiller 8.0.0 (Windows); modified using iText 4.2.0 by 1T3XT
    Subject: Am J Sports Med 0.0:03635465221128261
    Title: Clinical Outcomes After Combined ACL and Anterolateral Ligament Reconstruction Versus Isolated ACL Reconstruction With Bone–Patellar Tendon–Bone Grafts: A Matched-Pair Analysis of 2018 Patients From the SANTI Study Group
    Number of pages: 9
    
['\nClinical Outcomes After Combined ACL\nand Anterolateral Ligament ReconstructionVersus Isolated ACL Reconstruction WithBone–Patellar Tendon–Bone Grafts\nA Matched-Pair Analysis of 2018 Patients\nFrom the SANTI Study Group\nCharles Pioger, *MD , Lampros Gousopoulos, *MD, Graeme P. Hopper,yMD ,\nThais Dutra

# **BACKUP FUNCTION**

In [16]:
def EXTRACT_TABLE (res_part) :
    res=[]
    label=[]
    p=re.compile('TABLE [0-9]\n')
    pos=p.search(res_part)
    while pos!=None :
        label.append(res_part[pos.start():pos.end()])
        next=p.search(res_part[pos.end():])
        if next!=None :
            res.append(res_part[pos.end():next.start()+pos.end()])
        else :
            d=re.compile('[0-9]\n[^0-9]{40,100}')
            next=d.search(res_part[pos.end():])
            res.append(res_part[pos.end():next.start()+pos.end()+1])    
        res_part=res_part[pos.end():]
        pos=p.search(res_part)
    return(label,res)

In [17]:
def extract_information(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PdfReader(f)
        meta = reader.metadata
        number_of_pages = len(reader.pages)
        
        pages_text = []
        for n in range(number_of_pages):
            page = reader.pages[n]
            pages_text.append('\n'+page.extract_text())

    metadata = f"""
    Information about {pdf_path}: 

    Author: {meta.author}
    Creator: {meta.creator}
    Producer: {meta.producer}
    Subject: {meta.subject}
    Title: {meta.title}
    Number of pages: {number_of_pages}
    """
    g=re.compile('TABLE ')
    print(metadata)
    i=0
    tab=''
    while i < len(pages_text) :
        pos = g.search(pages_text[i])
        if pos!=None:
            tab=pages_text.pop(i)
            i=len(pages_text)
        else :
            i+=1
    return pages_text,metadata,tab

# **WORD PROCESS SYNOPSIS**

In [18]:
!pip install python-docx
import docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 879.0 kB/s eta 0:00:00


In [19]:
# Chemin vers le document Word
file_path = '/kaggle/input/synopsis-1/2024.06.24_Synopsis_GODENECHE_Format_HDH_VF.docx'

# Ouvrir le document Word
doc = docx.Document(file_path)

tables_text = []
for table in doc.tables:
    for row in table.rows:
        row_text = [cell.text for cell in row.cells]
        tables_text.append(row_text)

In [20]:
def SUBCOREDOCX(par) :
    res=[]
    label=['Introduction']
    p=re.compile('\n.{5,20}[\\xa0:]*\n')
    pos=p.search(par)
    prems=0
    while pos!=None :
        prems+=1
        res.append(par[:pos.start()])
        label.append(par[pos.start():pos.end()])
        par=par[pos.end():]
        pos=p.search(par)   
    res.append(par)
    if prems==0 :
        label=['Text Body']
    return (label,res)

In [21]:
print(SUBCOREDOCX(tables_text[6][1]))

(['Introduction', '\nObjectifs\n'], ['Contexte\n\nLa rupture du tendon grand pectoral est une pathologie rare. Dans la littérature, il est globalement décrit deux façons de prendre en charge chirurgicalement les ruptures du grand pectoral :\nSoit à un stade aigu (<6 semaines du traumatisme) : réinsertion directe du tendon au niveau du footprint.\nSoit à un stade chronique (au-delà de 6 semaines par rapport au traumatisme) : utilisation de greffe (auto/allo) pour pallier la rétraction du tendon.\nLe Dr Godenèche a privilégié une nouvelle approche\xa0chez des patients avec une rupture du tendon grand pectoral : une suture directe au footprint sans recours à la greffe.\n', '\nLes objectifs sont de décrire, dans chacun des deux groupes de patients (stade aigu et stade chronique)\xa0: \nLes caractéristiques des patients (âge, type de traumatisme, délai entre le traumatisme et la chirurgie, prise de stéroïdes, emploi, sport pratiqué et niveau de pratique) \nLe retour à l’emploi (type d’emplo

In [22]:
for para in tables_text:
    print(para)
    print("-------")

['', 'Résumé de l’étude, recherche ou évaluation\nà l’attention du porteur de projet\nObligatoirement en français – 4 pages maximum']
-------
['Titre/Acronyme\n(sera rendu public)', 'Evaluation d’une suture directe au footprint sans recours à la greffe chez des patients ayant une rupture du tendon grand pectoral']
-------
["PRÉSENTATION DE L'ÉQUIPE PROJET", "PRÉSENTATION DE L'ÉQUIPE PROJET"]
-------
['Responsable de traitement (RT)\nMerci de préciser :\nL’organisme d’appartenance  \nLe nom du représentant juridique et sa fonction\nLe nom du responsable scientifique et sa fonction si différent, et l’équipe associée\nLe comité Scientifique (le cas échéant)\nDélégué à la protection des données – DPD (le cas échéant)', 'Organisme d’appartenance\xa0: \nClinique Santy\nReprésentant juridique et sa fonction\xa0:\nArnaud Godenèche, chirurgien\nResponsable scientifique\xa0:\nArnaud Godenèche \nComité scientifique\xa0:\nNA\nDPO\xa0:\nArnaud Godenèche \n']
-------
['Responsable de la mise en œuvr

In [23]:
from docx import Document
def read_docx(file_path):
    doc = Document(file_path)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
  
print(read_docx('/kaggle/input/synopsis-1/2024.06.24_Synopsis_GODENECHE_Format_HDH_VF.docx'))